In [72]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [73]:
import h5py
import numpy as np
import os, sys, time
import multiprocessing
from ont_fast5_api.conversion_tools.conversion_utils import get_fast5_file_list

from cytosine.utils.fast5_helper import fast5_rescale, fast5_normalize_signal, raw_fast5
from cytosine.utils.extract_helper import save_feature_to_file, put_file_in_queue, get_ref_length, extract_preprocess, extract_perbase_signal_feature, get_mod_site_in_ref, get_batch_feature
from cytosine.extract_module import extract_fast5_feature

# Pre-precess the data

The fast5 file is copied from /fastscratch/c-panz/2021-09-30/guppy5.0.14_supmodel_tombo/T4LambdaTF1/0/workspace

input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/

00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5

000006ea-dddb-429c-8277-a81ce50da6a0.fast5

# Test fast5_helper

In [2]:
input_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
fast5_fn='00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5'

signal_group='Raw/Reads'
corrected_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl

In [3]:
# Test fast5 class
path = os.path.join(input_path, fast5_fn)
print(path)
fast5 = raw_fast5(path, corrected_group, basecall_group, basecall_subgroup, signal_group)
# Extract signal
readid, fast5_signal = fast5.fast5_signal() #get readid

#Extract event information: #Raw signal --> Normalization
event = fast5.fast5_event()
raw_signal = fast5_rescale(path, fast5_signal)
norm_signal = fast5_normalize_signal(raw_signal,  method='mad')

basecalled_seq, raw_signal_list, norm_signal_list = "", [], []
for e in event:
#    print(e)
    basecalled_seq += e[2]
#    print(norm_signal[e[0]:(e[0] + e[1])])
    norm_signal_list.append(norm_signal[e[0]:(e[0] + e[1])]) #event start position: end position(start+length)
    raw_signal_list.append(raw_signal[e[0]:(e[0] + e[1])])
    assert len(norm_signal_list) == len(raw_signal_list)
    
# Extract other information
chrom, align_strand, start, read_strand = fast5.fast5_align()
chrom, align_strand, start, read_strand

/pod/2/li-lab/Ziwei/Nanopore/daily/test/test/00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5


('chr4', '-', 118511944, 't')

# Test extract_helper 
Function: get_ref_length, get_mod_site_in_ref, extract_perbase_signal_feature, get_batch_feature

In [4]:
# get_ref_length
ref_path = '/pod/2/li-lab/Ziwei/Nanopore/data/reference/hg38.fa'
ref_total_length = get_ref_length(ref_path)

In [5]:
#get_mod_site_in_ref
mod_site_loc = get_mod_site_in_ref(basecalled_seq, motif_seq ='CG', mod_loc = 0)
len(mod_site_loc)

11816

### Extract_perbase_signal_feature
Raw signal --> Normalization --> alignment --> methylated site --> features

### put_file_in_queue

In [10]:
fast5_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
fast5_q = multiprocessing.Queue()
batch_size = 100

fast5_files = get_fast5_file_list(fast5_path, recursive=True)
put_file_in_queue(fast5_q, fast5_files, batch_size)

In [11]:
fast5_q.empty()

False

## extract_preprocess

In [12]:
ref_length, fast5_q, num_fast5 = extract_preprocess(fast5_path, ref_path, batch_size)

Find 3 fast5 files!
Read genome reference..


In [13]:
fast5_q.empty()

False

### get_batch_feature

In [5]:
#Get fast5_q
fast5_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
ref_path = '/pod/2/li-lab/Ziwei/Nanopore/data/reference/hg38.fa'
batch_size = 100
ref_length, fast5_q, num_fast5 = extract_preprocess(fast5_path, ref_path, batch_size)

Find 3 fast5 files!
Read genome reference..


In [6]:
fast5_q.empty()

False

### extract_perbase_signal_feature

In [60]:
fast5_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
ref_path = '/pod/2/li-lab/Ziwei/Nanopore/data/reference/hg38.fa'
output_file='/pod/2/li-lab/Ziwei/Nanopore/daily/2021-10-12/feature_test.csv'

num_proc = 1

corrected_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl
signal_group='Raw/Reads'
normalize_method = 'mad'
motif_seq = 'CG'
mod_loc = 0
ref_total_length = get_ref_length(ref_path)
kmer=17
mod_label=1

ref_length, fast5_q, num_fast5 = extract_preprocess(fast5_path, ref_path, batch_size)

Find 3 fast5 files!
Read genome reference..


In [61]:
fast5_q.empty()

False

In [62]:
fast5_num = 0

    #Extract features around CG in each read
while not fast5_q.empty():
    #https://stackoverflow.com/a/63234109
    try:
        fast5_files = fast5_q.get()
#        print(fast5_files, type(fast5_files))
    except Exception:
        break
            
    fast5_num += len(fast5)
    feature_list, error_num = extract_perbase_signal_feature(fast5_files, 
                                                             corrected_group, basecall_group, basecall_subgroup, signal_group, normalize_method, 
                                                             motif_seq, mod_loc, ref_total_length, kmer, mod_label)

['/pod/2/li-lab/Ziwei/Nanopore/daily/test/test/000006ea-dddb-429c-8277-a81ce50da6a0.fast5', '/pod/2/li-lab/Ziwei/Nanopore/daily/test/test/00047d2f-1825-4151-a702-f01d559f22ab.fast5', '/pod/2/li-lab/Ziwei/Nanopore/daily/test/test/00000156-e575-4fb7-9053-d00dbe5c8d9c.fast5'] <class 'list'>
File 00047d2f-1825-4151-a702-f01d559f22ab.fast5 didn't have event info

File 00047d2f-1825-4151-a702-f01d559f22ab.fast5 didn't have alignment info


In [63]:
len(feature_list)

19969

### Get batch feature

In [74]:
fast5_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
ref_path = '/pod/2/li-lab/Ziwei/Nanopore/data/reference/hg38.fa'
output_file='/pod/2/li-lab/Ziwei/Nanopore/daily/2021-10-12/feature_test.csv'

num_proc = 1
batch_size = 100

corrected_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl
signal_group='Raw/Reads'
normalize_method = 'mad'
motif_seq = 'CG'
mod_loc = 0
ref_total_length = get_ref_length(ref_path)
kmer=17
mod_label=1

In [75]:
ref_length, fast5_q, num_fast5 = extract_preprocess(fast5_path, ref_path, batch_size)

feature_q = multiprocessing.Queue()
error_q = multiprocessing.Queue()

Find 3 fast5 files!
Read genome reference..


In [76]:
fast5_q.empty()

False

In [54]:
get_batch_feature(fast5_q, feature_q, error_q,  
                  corrected_group, basecall_group, basecall_subgroup, signal_group, 
                  normalize_method, motif_seq, mod_loc,ref_total_length, kmer, mod_label)

File 00047d2f-1825-4151-a702-f01d559f22ab.fast5 didn't have event info

File 00047d2f-1825-4151-a702-f01d559f22ab.fast5 didn't have alignment info
Batch feature extraction is done for 3 fast5s


In [51]:
feature_q.empty()

True

### save_feature_to_file

In [55]:
output_file = '/pod/2/li-lab/Ziwei/Nanopore/daily/2021-10-12/test.tsv'

In [67]:
import time
time_wait = 20
def save_feature_to_file(output_file, feature_q):
    if os.path.exists(output_file) and os.path.isfile(output_file):
        raise FileExistError("{} already exist. Please provide a new file name with path".format(output_file))
    else:
        with open(output_file, 'w') as output_fp:
            while True:
                if feature_q.empty():
                    time.sleep(time_wait)
                    continue
                feature_list = feature_q.get()
                    
                for feature in feature_list:
                    output_fp.write(feature, "\n")
                output_fp.flush()

In [68]:
save_feature_to_file(output_file, feature_q)

KeyboardInterrupt: 

# extract_fast5_feature

In [112]:
fast5_path='/pod/2/li-lab/Ziwei/Nanopore/daily/test/test'
ref_path = '/pod/2/li-lab/Ziwei/Nanopore/data/reference/hg38.fa'
output_file='/pod/2/li-lab/Ziwei/Nanopore/daily/2021-10-12/feature_test.csv'

num_proc = 1

corrected_group='RawGenomeCorrected_001' #tombo resquiggle save location
basecall_group='Basecall_1D_001' #has to be save in the tombo requiggle step
basecall_subgroup='BaseCalled_template' #Attention: the basecall_subgroup can be 'basecalled_compl
signal_group='Raw/Reads'
normalize_method = 'mad'
motif_seq = 'CG'
mod_loc = 0
ref_total_length = get_ref_length(ref_path)
kmer=17
mod_label=1

In [152]:
ref_total_length, fast5_q, num_fast5 = extract_preprocess(fast5_path, ref_path, batch_size)
    
feature_q = multiprocessing.Queue()
error_q = multiprocessing.Queue()
    
print('Extracting features from nanopore reads...')
    
#Extract feature
feature_procs = []
    
if num_proc > 1:
    num_proc -= 1

for _ in range(num_proc):
    # get batch feature from single read
    p = multiprocessing.Process(
        target=get_batch_feature, 
        args=(fast5_q, feature_q, error_q,  
              corrected_group, basecall_group, basecall_subgroup, signal_group, normalize_method,  
              motif_seq, mod_loc, ref_total_length, kmer, mod_label)
    )
    p.aeomon = True
    p.start()
    feature_procs.append(p)

Find 3 fast5 files!
Read genome reference..
Extracting features from nanopore reads...
Batch feature extraction is done for 0 fast5s
Batch feature extraction is done for 0 fast5s
File 00047d2f-1825-4151-a702-f01d559f22ab.fast5 didn't have event info

File 00047d2f-1825-4151-a702-f01d559f22ab.fast5 didn't have alignment info


In [115]:
fast5_q

In [113]:
fast5_list = [os.path.join(fast5_path, x) for x in os.listdir(fast5_path)]